# 무신 카페 검색 & 예약 플러그인 예제(LangChain)
서로 다른 2개의 툴을 ChatGPT 플러그인으로 공개하고, AI 오케스트레이터인 LangChain에서 호출하는 시스템을 구현하는 예제다. '최충헌과 관련된 카페의 이름을 검색하고, 7월 1일 18시에 예약할 수 있는지 알려줘'와 같이 여러 작업을 한 번의 지시로 해결할 수 있는 시스템을 구축할 것이다.

![](./images/04_001.png)


# 사전 준비

이 파이썬 예제를 실행하려면 다음과 같은 환경이 필요하다:

- Azure OpenAI Service를 사용할 수 있는 [승인 완료](https://aka.ms/oai/access)된 Azure 구독
- Azure OpenAI Service 연동 및 모델 정보
  - OpenAI API 키
  - OpenAI `gpt-4` 혹은 `gpt-4o` 모델의 배포 이름
  - OpenAI API 버전
- Python (이 예제는 버전 3.12.4로 테스트 했다.)

<div class="alert alert-block alert-success"><b>Note:</b> 고도화된 Reasoning을 정확도 높게 수행하려면 GPT-4o 모델을 사용하는 것을 권장한다.</div>

이 예제에서는 Visual Studio Code와 [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)를 사용한다.


## 설정

다음 플러그인들을 실행시켜 놓는다.
- cafe-review-plugin
- restaurant-reservation-plugin

각각의 플러그인 디렉토리(`plugins/`)에 들어가 아래 커맨드를 실행한다.

```bash
pip install -r requirements.txt
```

라이브러리 설치가 완료되면

```bash
python main.py
```

로 서버를 실행한다.

## 패키지 설치

In [ ]:
!pip install openai==1.35.3
!pip install langchain==0.2.5
!pip install langchain-openai==0.1.9
!pip install langchain-community==0.2.5

In [ ]:
import openai
print("openai", openai.__version__)
import langchain
print("langchain", langchain.__version__)

## 라이브러리 및 환경변수 불러오기

In [ ]:
# from langchain_openai import ChatOpenAI #If you use OpenAI's API
from langchain_openai import AzureChatOpenAI
from langchain import hub
from langchain.agents import (
    load_tools,
    AgentExecutor,
    create_react_agent
)
from langchain_community.tools import AIPluginTool

from requests.exceptions import ConnectionError

## Azure OpenAI 설정

In [ ]:
AZURE_OPENAI_API_KEY = "Your OpenAI API Key"
AZURE_OPENAI_ENDPOINT = "https://<Your OpenAI Service>.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "gpt-4o"

In [ ]:
# If you use OpenAI's API
#import os
#os.environ["OPENAI_API_KEY"] = "<Your OpenAI KEY>"

# Tool 불러오기
ChatGPT Plugin을 불러오기 위해 AIPluginTool에 등록한다.

In [ ]:
# If you use OpenAI's API
# llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)

llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_API_KEY,
    # azure_ad_token_provider=self.openai_ad_token,
    temperature=0.0,
)
tools = load_tools(["requests_all"])
plugin_urls = [
    "http://localhost:5005/.well-known/ai-plugin.json",
    "http://localhost:5006/.well-known/ai-plugin.json",
]

tools += [AIPluginTool.from_plugin_url(url) for url in plugin_urls]

# Agent 초기화

[ReAct 에이전트](https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html)를 사용한다. 제약조건은 SUFFIX에 추가한다.


In [ ]:
SUFFIX = """
'Answer should be in Korean. Use http instead of https for endpoint.
If there is no year in the reservation, use the year 2023. 
"""
prompt = hub.pull("hwchase17/react")

# Responsible AI MetaPrompt
# **IMPORTANT**
# If a restaurant reservation is available, must check with the user before making a reservation if yes.'

agent = create_react_agent(llm, tools, prompt + SUFFIX)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate"
)

# 실행
오래 기다려도 응답이 오지 않을 때는, GPT-4o 모델을 사용하면 성공 확률을 높일 수 있다. GPT-35-Turbo는 시스템 메시지를 잘 작성해야 성공 확률이 높아진다.

In [ ]:
try:
    result = agent_executor.invoke({"input": "최충헌과 관련된 카페의 이름을 검색해줘."})
    print(result)
except ConnectionError as e:
    print("죄송합니다. 잘 모르겠습니다(ConnectionError).", e)
except Exception as e:
    print("죄송합니다. 잘 모르겠습니다(Error).", e)

In [ ]:
try:
    result = agent_executor.invoke({"input": "최충헌과 관련된 카페의 이름을 검색하고, 7월 1일 18시에 예약할 수 있는지 알려줘. 예약이 가능하면 예약해줘."})
    print(result)
except ConnectionError as e:
    print("죄송합니다. 잘 모르겠습니다(ConnectionError).", e)
except Exception as e:
    print("죄송합니다. 잘 모르겠습니다(Error).", e)


In [ ]:
try:
    result = agent_executor.invoke({"input": "Escape room 카페에 7월 1일 18시로 예약해줘."})
    print(result)
except ConnectionError as e:
    print("죄송합니다. 잘 모르겠습니다(ConnectionError).", e)
except Exception as e:
    print("죄송합니다. 잘 모르겠습니다(Error).", e)
